In [ ]:
import findspark

In [ ]:
findspark.init('/home/nahle/spark-2.1.0-bin-hadoop2.7')

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("NN").getOrCreate()

In [ ]:
data = spark.read.format('libsvm').load('simpleNN.txt')

In [ ]:
data.show(truncate = False)

In [ ]:
data.select('label').distinct().show()

In [ ]:
data.groupBy('label').count().show()

In [ ]:
train, test = data.randomSplit([0.7, 0.3])

In [ ]:
train.cache()

In [ ]:
test.cache()

# Let's train a Neural Network

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [ ]:
layers = [4, 5, 4, 3]

In [ ]:
classifier = MultilayerPerceptronClassifier(maxIter=100,
                                           layers= layers)

In [ ]:
classifier.explainParam('solver')

In [ ]:
classifier.explainParam('maxIter')

In [ ]:
model = classifier.fit(train)

In [ ]:
preds = model.transform(test)

In [ ]:
preds.show(truncate = False)

In [ ]:
predictionsAndLabels = preds.select(['prediction', 'label'])

In [ ]:
predictionsAndLabels.show()

# Evaluating classifiers: Confusion Matrix

![Confusion Matrix](images/confusionMatrix.png)

## Classification Metrics
- __Accuracy__ : measures the number of data points classified correctly. Accuracy is not a good metric for skewed classes  (for instance, 90 % of the labels in the training set are class A, 10 % class B)!!

$$Accuracy=\frac{TP+TN}{TP+TN+FP+FN} $$

Where:

    -TP: Number of True Positives
    -FP: Number of False Positives
    -TN: Number of True Negatives
    -FN: Number of False Negatives

- __Precision__: Out of all the examples the classifier labeled as positive, what fraction were correct?

$$Precision=\frac{TP}{TP+FP} $$

- __Recall__: "Out of all the positive examples, what fraction did the classifier pick up?"

$$Recall=\frac{TP}{TP+FN} $$

- __F1 score__: It is the harmonic average of the precision and recall. It's best value is 1 (when both are 1). Its worst value is 0. 

$$F1= 2*\frac{Precision * Recall}{Precision + Recall} $$

## More on:
http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName='f1') #  

In [ ]:
evaluator.explainParam('metricName')

In [ ]:
evaluationNN = evaluator.evaluate(predictionsAndLabels)

In [ ]:
print ('NeauralNetwork F1 = %g'%evaluationNN)

In [ ]:
model.layers


In [ ]:
model.weights

In [ ]:
len(model.weights)

# Model Tuning

Spark provides 2 efficient classes that facilitate model tuning: **ParamGridBuilder and CrossValidator**:

More on: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.tuning
    

## Pipeline
- A pipeline acts as an estimator. 
- A Pipeline consists of a sequence of stages, each of which is either an Estimator or a Transformer. 
- When Pipeline.fit is called, the stages are executed in order. 
- If a stage is an Estimator its Estimator.fit method will be called on the input dataset to fit a model. Then the model, which is a transformer, will be used to transform the dataset as the input to the next stage. 
- If a stage is a Transformer, its Transformer.transform method will be called to produce the dataset for the next stage.

## ParamGridBuilder

Builder for a param grid used in grid search-based model selection

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
#help(Pipeline)

In [ ]:
#help(CrossValidator)

In [ ]:
pipeLine=Pipeline()
pipeLine.setStages([classifier])

In [ ]:
gridBuilder = ParamGridBuilder().addGrid(classifier.layers, 
                                         [[4, 2, 3, 3], [4, 2, 2, 3], [4, 10, 3]]
                                        ).addGrid(classifier.stepSize, [0.03, 0.01]).build()

In [ ]:
classifier.explainParam('stepSize')

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
cv = CrossValidator(estimator=pipeLine, 
                    estimatorParamMaps=gridBuilder, 
                    evaluator=evaluator)

In [ ]:
cvm = cv.fit(train)

In [ ]:
predictions = cvm.transform(test)

In [ ]:
evaluator.evaluate(predictions)

# Let's see which model was the best !

In [ ]:
bestModel = cvm.bestModel.stages[0]

In [ ]:
bestModel.weights.array

In [ ]:
bestModel.layers

In [ ]:
cvm.avgMetrics


# Now, try other classifiers:

To learn about the different classifiers implemented in Spark:

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.classification

Next, we will train and evaluation 3 among them:
   1. LogisticRegressior
   1. DecisionTreeClassifier
   1. RandomForestClassifier

In [ ]:
from pyspark.ml.classification import (LogisticRegression, 
                                       DecisionTreeClassifier, 
                                       RandomForestClassifier
                                      )

In [ ]:
lr  = LogisticRegression()
dt = DecisionTreeClassifierœ()
rf = RandomForestClassifier(maxBins=10)


In [ ]:
dt.explainParams()

In [ ]:
lrm = lr.fit(train)

In [ ]:
dtm = dt.fit(train)

In [ ]:
rfm = rf.fit(train)

In [ ]:
lrm_results = lrm.transform(test)
dtm_results = dtm.transform(test)
rfm_results = rfm.transform(test)


In [ ]:
evaluation_lrm=evaluator.evaluate(lrm_results)
evaluation_dtm=evaluator.evaluate(dtm_results)
evaluation_rfm=evaluator.evaluate(rfm_results)

In [ ]:
print('evaluation of logistic regression model = %g'%evaluation_lrm)
print('evaluation of decision tree model = %g'%evaluation_dtm)
print('evaluation of random forest model = %g'%evaluation_rfm)